# Projeto IA 1ªParte a22007545 - Edith Sousa; a21900295 - Frederico Azevedo; a21906959 - Nuno

### Library import

In [52]:
import csv as csv
import numpy as np


### Algoritmo de pesquisa informada num ambiente estático com obstáculos. O objetivo será encontrar o caminho de menor custo desde o nó de partida até ao nó final.

In [57]:
class Game():
    def __init__(self):
        self.initialize_game()

    def initialize_game(self):

        # Podemos ler do ficheiro S ; E ; T ; A ; B ; F
        # Agua (A) tem de ser obrigatoriamente 2x2
        # Paredes (B) tem de ser obrigatoriamente 1x10 ou 10x1
        # Fronteiras(F)  Delimita todo o ambiente
        print('oi')
        # with open('sample-environment.csv', newline='') as csvfile:
        #     file = csv.reader(csvfile, delimiter=' ', quotechar='|')
        #     for row in file:
        #         print(row)

                


In [58]:
def main():
    g = Game()
    

if __name__ == "__main__":
    main()

oi
